In [ ]:
#PRIMA CELLA NON UTILE PERCHE' QUESTO APPROCCIO BASATO SULLA RETE WORD2VEC E' STATO POI ABBANDONATO IN FAVORE DELL'USO DI
#BAG OF WORDS CHE E' PIU' SEMPLICE, PIU' GENERALIZZABILE E OFFRE PERFORMANCE SOLO DI POCO INFERIORI

"""
!git clone https://github.com/facebookresearch/fastText.git
%cd fastText
!sudo pip install .
import fasttext
import fasttext.util
fasttext.util.download_model('it', if_exists='ignore')
ft = fasttext.load_model('cc.it.300.bin')
print(ft.get_dimension())

similarity_materie_prime = []
similarity_materie_consumo = []
similarity_merci = []
similarity_merci_prodotti = []

vector_formaggio = ft.get_word_vector('formaggio')
vector_uova = ft.get_word_vector('uova')
vector_busta = ft.get_word_vector('busta')
vector_posate = ft.get_word_vector('posate')
vector_kinder = ft.get_word_vector('kinder')
vector_caramella = ft.get_word_vector('caramella')
vector_bibita = ft.get_word_vector('bibita')
vector_vodka = ft.get_word_vector('vodka')

for s in list(descr):
  s = s.lower()
  s_words = s.split(" ")

  words_similarities_materie_prime = []
  words_similarities_materie_consumo = []
  words_similarities_merci = []
  words_similarities_merci_prodotti = []

  for i in s_words:
    vector_i = ft.get_word_vector(i)
    score_materie_prime = ((1 - spatial.distance.cosine(vector_i, vector_formaggio)) + (1 - spatial.distance.cosine(vector_i, vector_uova)))/2
    score_materie_consumo = ((1 - spatial.distance.cosine(vector_i, vector_busta)) + (1 - spatial.distance.cosine(vector_i, vector_posate)))/2
    score_merci = ((1 - spatial.distance.cosine(vector_i, vector_kinder)) + (1 - spatial.distance.cosine(vector_i, vector_caramella)))/2
    score_merci_prodotti = ((1 - spatial.distance.cosine(vector_i, vector_bibita)) + (1 - spatial.distance.cosine(vector_i, vector_vodka)))/2

    words_similarities_materie_prime.append(score_materie_prime)
    words_similarities_materie_consumo.append(score_materie_consumo)
    words_similarities_merci.append(score_merci)
    words_similarities_merci_prodotti.append(score_merci_prodotti)

  words_similarities_materie_prime = [x for x in words_similarities_materie_prime if pd.notnull(x)]
  words_similarities_materie_consumo = [x for x in words_similarities_materie_consumo if pd.notnull(x)]
  words_similarities_merci = [x for x in words_similarities_merci if pd.notnull(x)]
  words_similarities_merci_prodotti = [x for x in words_similarities_merci_prodotti if pd.notnull(x)]

  best_materie_prime = max(words_similarities_materie_prime)
  best_materie_consumo = max(words_similarities_materie_consumo)
  best_merci = max(words_similarities_merci)
  best_merci_prodotti = max(words_similarities_merci_prodotti)

  similarity_materie_prime.append(best_materie_prime)
  similarity_materie_consumo.append(best_materie_consumo)
  similarity_merci.append(best_merci)
  similarity_merci_prodotti.append(best_merci_prodotti)
"""

In [1]:
#NOTEBOOK 1: data cleaning, feature selection e fitting di modelli di classificazione, con analisi delle performance e degli errori commessi.
#La prima cella è dedicata a settare la rete Word2Vec in italiano che serve a estrarre features semantiche dalle descrizioni 
#delle fatture in modo da distinguere meglio le classi

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#lettura in pandas del file excel che contiene la relazione input-output matchando fatture XML e prima nota

import pandas as pd

data = pd.read_excel('/content/drive/MyDrive/df_final.xlsx')
data = data.drop('Unnamed: 0', axis=1)
#print(data.head(5))
print(data.columns)
print(data.shape)

Index(['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceValore_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_ScontoMaggiorazione_Tipo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_ScontoMaggiorazione_Percentuale_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_ScontoMaggiorazione_Importo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AltriDatiGestionali_TipoDato_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AltriDatiGestionali_RiferimentoTesto_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AltriDatiGestionali_RiferimentoNumero_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AltriDatiGestionali_RiferimentoData_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_NumeroLinea',
       ...
    

In [3]:
#selezione delle colonne di interesse

data = data[['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_Descrizione',
            'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale',
            'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA',
            'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice',
            'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data',
            'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT',
            'Conto']]

print(data.shape)
#print(data.head(5))

(5877, 11)


In [5]:
#controllo del tipo delle colonne

for column in data.columns:
  print(column)
  print(type(data[column][0]))

FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario
<class 'numpy.float64'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale
<class 'numpy.float64'>
FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first
<class 'str'>
FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA
<class 'numpy.float64'>
FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice
<class 'numpy.int64'>
FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data
<class 'pandas._libs.tslibs.timestamps.Timestamp'>
FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT
<class 'str'>
Conto
<class 'str'>


In [4]:
#conversione del tipo di dato in alcune colonne

from datetime import datetime

data['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA'] = data['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA'].astype(int)
data['FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice'] = data['FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice'].astype(str)


In [5]:
#conversione della data in 3 colonne giorno-mese-anno per renderle feature adatte a fare machine learning, e
#conseguente eliminazione della colonna con la data in formato timestamp

year = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).year)
month = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).month)
day = pd.Series(pd.DatetimeIndex(data['FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data']).day)

frame = {'year': year, 'month': month, 'day': day}
data2 = pd.DataFrame(frame)
  
data = pd.concat(objs=[data, data2], axis=1)
data.drop('FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Data', axis=1, inplace=True)

print(data.columns)

Index(['FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_Descrizione',
       'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale',
       'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first',
       'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA',
       'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice',
       'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT', 'Conto',
       'year', 'month', 'day'],
      dtype='object')


In [6]:
#rinominazione delle colonne

data = data.rename(columns={'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_Descrizione': 'Descrizione',
            'FatturaElettronicaBody_DatiGenerali_DatiGeneraliDocumento_Numero': 'Numero',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_CodiceArticolo_CodiceTipo_first': 'CodiceTipo',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoUnitario': 'PrezzoUnitario',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_PrezzoTotale': 'PrezzoTotale',
            'FatturaElettronicaBody_DatiPagamento_DettaglioPagamento_ModalitaPagamento_first': 'ModalitaPagamento',
            'FatturaElettronicaBody_DatiBeniServizi_DettaglioLinee_AliquotaIVA': 'AliquotaIVA',
            'FatturaElettronicaHeader_CedentePrestatore_DatiAnagrafici_IdFiscaleIVA_IdCodice': 'CodiceCedente',
            'FatturaElettronicaBody_DatiGenerali_DatiDDT_NumeroDDT': 'NumeroDDT',
            'Conto': 'Conto', 'year': 'Year', 'month': 'Month', 'day': 'Day'})

print(data.columns)

Index(['Descrizione', 'Numero', 'CodiceTipo', 'PrezzoUnitario', 'PrezzoTotale',
       'ModalitaPagamento', 'AliquotaIVA', 'CodiceCedente', 'NumeroDDT',
       'Conto', 'Year', 'Month', 'Day'],
      dtype='object')


In [7]:
#eliminazione, in tutte le colonne tranne 'Conto', dei nan che compaiono come float a favore di stringhe vuote 
#(perche tutte le colonne di feature con dei nan sono in formato stringa)

Conti = data.Conto
data = data.fillna('')
data.Conto = Conti
print(data.isnull().sum().sum())


4061


In [8]:
#semplificazione N1: tutte le features categoriche vengono gestite col semplice One Hot Encoding, anche se questo causa
#un grande aumento del numero di features. Altri encoders potrebbero performare meglio (es: hashing trick).
#semplificazione N2: non si considera la struttura gerarchica dei conti ma le labels vengono predette come fossero 13 classi indipendenti
#semplificazione N3: si valutano le performance con una semplice divisione train-test senza ricorrere alla CV (sarà poi fatto nel Notebook 2)
#semplificazione N4: non si risistemano le stringe del CodiceTipo che potrebbero essere formattate meglio dando vita a meno
#classi binarie

#N1
ModalitaPagamento_encoded = pd.get_dummies(data.ModalitaPagamento, prefix='ModalitaPagamento')
print(ModalitaPagamento_encoded.shape)
CodiceTipo_encoded = pd.get_dummies(data.CodiceTipo, prefix='CodiceTipo')
print(CodiceTipo_encoded.shape)
NumeroDDT_encoded = pd.get_dummies(data.NumeroDDT, prefix='NumeroDDT')
print(NumeroDDT_encoded.shape)
CodiceCedente_encoded = pd.get_dummies(data.CodiceCedente, prefix='CodiceCedente')
print(CodiceCedente_encoded.shape)

data.drop(['CodiceTipo', 'ModalitaPagamento', 'CodiceCedente', 'NumeroDDT'], axis=1, inplace=True)
data = pd.concat(objs=[data, CodiceTipo_encoded, ModalitaPagamento_encoded, CodiceCedente_encoded, NumeroDDT_encoded], axis=1)

print(data.shape)
print(data.columns)

#si arriva ad avere 203 features, a causa dei One Hot Encoding

(5877, 8)
(5877, 23)
(5877, 109)
(5877, 57)
(5877, 206)
Index(['Descrizione', 'Numero', 'PrezzoUnitario', 'PrezzoTotale',
       'AliquotaIVA', 'Conto', 'Year', 'Month', 'Day', 'CodiceTipo_',
       ...
       'NumeroDDT_PKL.21.1321', 'NumeroDDT_PKL.21.1457',
       'NumeroDDT_PKL.21.161', 'NumeroDDT_PKL.21.195', 'NumeroDDT_PKL.21.471',
       'NumeroDDT_PKL.21.520', 'NumeroDDT_PKL.21.55', 'NumeroDDT_PKL.21.597',
       'NumeroDDT_PKL.21.914', 'NumeroDDT_PKL.21.97'],
      dtype='object', length=206)


In [ ]:
#analisi della distribuzione della variabile target

print(data.groupby('Conto').count())
print(data.shape)

In [ ]:
#rimozione delle unità statistiche con target troppo rari (threshold = 5, incluso). La molteplicità dei target scende da 24 a 13.
#per farlo si crea un nuovo df per non perdere traccia delle unità statistiche eliminate

mask = (data['Conto'] != '18/40/501')&(data['Conto'] != '66/25/505')&(data['Conto'] != '66/25/508')&(data['Conto'] != '66/30/060')&(data['Conto'] != '68/05/005')&(data['Conto'] != '68/05/133')&(data['Conto'] != '68/05/290')&(data['Conto'] != '68/05/320')&(data['Conto'] != '68/05/385')&(data['Conto'] != '68/05/407')&(data['Conto'] != '88/20/035')
data2 = data[mask]
data2.reset_index(inplace=True)
data2 = data2.drop(['index'], axis=1)
print(data2)

In [11]:
#in vista del semi supervised learning, separazione dei dati supervisionati e non supervisionati

from sklearn.semi_supervised import SelfTrainingClassifier

data_labeled = data2[data2['Conto'].notna()]
data_unlabeled = data2[data2['Conto'].isnull()]

data_labeled.reset_index(inplace=True)
data_labeled = data_labeled.drop('index', axis=1)
data_unlabeled.reset_index(inplace=True)
data_unlabeled = data_unlabeled.drop('index', axis=1)

print(data_labeled.shape)
print(data_unlabeled.shape)

#si usa solo la porzione di dati labeled in un primo train-test di alcuni classificatori

(1794, 206)
(4061, 206)


In [12]:
#divisione dei dati supervised in train e test set che saranno comuni a tutte le configurazioni e i modelli provati

from sklearn.model_selection import train_test_split
features = data_labeled.drop(['Descrizione', 'Conto', 'Numero'], axis=1)
targets = data_labeled.Conto
num = data_labeled.Numero
descr = data_labeled.Descrizione

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
print(X_train.shape)
print(X_test.shape)

(1255, 203)
(539, 203)


In [13]:
#trasformazione della descrizione in features numeriche grazie al metodo Bag-of-Words, implementato grazie alla classe
#CountVectorizer di sklearn. Ha più senso costruire il dizionario di parole adesso, dopo aver separato i dati unsupervised,
#perchè questi non saranno comunque usati nella procedura di train-test e ampliano solamente la dimensione del dizionario.
#Inoltre, per simulare più realisticamente la realtà operativa, il dizionario viene costruito usando solo il training set;
#le nuove parole presenti nel test set saranno ignorate.

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

descr_train = descr[X_train.index]
descr_test = descr[X_test.index]

print(descr.head())

vectorizer = CountVectorizer()
bag_train = vectorizer.fit_transform(list(descr_train))
bag_train = pd.DataFrame(bag_train.toarray(), columns=vectorizer.get_feature_names_out())
print(bag_train.head(10))
print(bag_train.shape)

bag_test = vectorizer.transform(list(descr_test))
bag_test = pd.DataFrame(bag_test.toarray(), columns=vectorizer.get_feature_names_out())
bag_test.index = X_test.index

print(bag_test.head(10))
print(bag_test.shape)
print(bag_train.index)
print(bag_test.index)
print(X_train.index)
print(X_test.index)

0    A513-77199580-8000500131329-NUTELLA G3000X2 SC...
1    CONTRIBUTO CONAI-A513-77199580-8000500131329-N...
2             AB3 MISCELA CREMA BAR CONFEZIONE DA Kg.3
3                                        /D Lotto V38I
4                  N ZUCCHERO SEMOLATO ASTUCCI da Kg.1
Name: Descrizione, dtype: object
   00  000  000764  000887  000899  000928  002  00217  00222  007694433  ...  \
0   0    0       0       0       0       0    0      0      0          0  ...   
1   0    0       0       0       0       0    0      0      0          0  ...   
2   0    0       0       0       0       0    0      0      0          0  ...   
3   0    0       0       0       0       0    0      0      0          0  ...   
4   0    0       0       0       0       0    0      0      0          0  ...   
5   0    0       0       0       0       0    0      0      0          0  ...   
6   0    0       0       0       0       0    0      0      0          0  ...   
7   0    0       0       0       0       0  

In [14]:
#estensione di X_train e X_test grazie alle nuove features estratte dalla descrizione. Estensione anche del df features,
#composto da questi 2, perchè serve per fare feature selection

X_train = pd.concat(objs=(X_train, bag_train), axis=1)
X_test = pd.concat(objs=(X_test, bag_test), axis=1)
features = pd.concat(objs=(X_train, X_test), axis=0)

print(X_train.shape)
print(X_test.shape)
print(features.shape)

#ora ogni sample ha oltre 1500 feature, la maggior parte delle quali provengono dalla descrizione testuale

(1255, 1561)
(539, 1561)
(1794, 1561)


In [260]:
#creazione del dataset totale (con tutte le righe di data) inserendo anche le features provenienti dalla descrizione (il dizionario
#comunque è quello creato con le sole righe di training). Questo è il dataset da cui ripartire nel Notebook 2

descr_all = data.Descrizione
bag_all = vectorizer.transform(list(descr_all))
bag_all = pd.DataFrame(bag_all.toarray(), columns=vectorizer.get_feature_names_out())

data = pd.concat(objs=(data, bag_all), axis=1)
print(data.shape)

data.to_excel('/content/df_preprocessed.xlsx')

(5877, 1564)


In [21]:
#primo classificatore provato: Naive Bayes, con diverse assunzioni sulla distribuzione delle features condizionata al target. 

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import BernoulliNB

gnb = BernoulliNB()
y_pred = gnb.fit(X_train, y_train).predict(X_test)
print(1-((y_test != y_pred).sum()/ X_test.shape[0]))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#accuracy performance: GaussianNB: 0.79, BernoulliNB: 0.83

0.7884972170686456
[0.61363636 0.66920152 0.8875     0.9057971  0.75       0.
 0.         0.         0.85714286 0.98275862 0.        ]
0.7884972170686455
0.5150942239846089
0.7765685183653288


In [16]:
#secondo classificatore provato: Decision Tree

from sklearn import tree
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score

clf = tree.DecisionTreeClassifier(max_depth=25)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))
#print(clf.feature_importances_)

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
#print(f1_score(y_test, y_pred, average='micro')), è inutile tanto è uguale all'accuracy
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#riduzione delle features secondo il criterio di importanza basato sul Gini 

model = SelectFromModel(clf, prefit=True)
features_new = model.transform(features)
print(features.shape)
print(features_new.shape)

X_train_new, X_test_new, y_train, y_test = train_test_split(features_new, targets, test_size=0.3, shuffle=False)
clf2 = tree.DecisionTreeClassifier(max_depth=25)
y_pred2 = clf2.fit(X_train_new, y_train).predict(X_test_new)
print(accuracy_score(y_test, y_pred2))
#print(clf2.feature_importances_)

#le performance sono intorno a 0.93 sia col modello completo che con quello ridotto, che ha circa 50 features

0.9257884972170687
[0.93495935 0.88505747 0.91358025 0.92366412 1.         1.
 0.8        1.         1.         1.         0.33333333]
0.8900540475674711
0.9296612910951791
(1794, 1561)
(1794, 49)
0.9332096474953617


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


In [ ]:
#plot colorato del decision tree

!pip install graphviz
import graphviz
import pydotplus
dot_data = tree.export_graphviz(clf, out_file=None, 
                                feature_names=features.columns,  
                                class_names=targets,
                                filled=True)

pydot_graph = pydotplus.graph_from_dot_data(dot_data)
pydot_graph.write_png('/content/original_tree.png')
pydot_graph.set_size('"5,5!"')
pydot_graph.write_png('/content/resized_tree.png')
pydot_graph
# Draw graph
#graph = graphviz.Source(dot_data, format="png") 
#graph


In [55]:
#terzo classificatore provato: Random Forest

from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

"""
scaler = StandardScaler()
features.PrezzoUnitario = scaler.fit_transform(np.array(features.PrezzoUnitario).reshape(-1, 1))
features.PrezzoTotale = scaler.fit_transform(np.array(features.PrezzoTotale).reshape(-1, 1))
features.AliquotaIVA = scaler.fit_transform(np.array(features.AliquotaIVA).reshape(-1, 1))
features.Year = scaler.fit_transform(np.array(features.Year).reshape(-1, 1))
features.Month = scaler.fit_transform(np.array(features.Month).reshape(-1, 1))
features.Day = scaler.fit_transform(np.array(features.Day).reshape(-1, 1))
features.Similarity_Materie_Prime = scaler.fit_transform(np.array(features.Similarity_Materie_Prime).reshape(-1, 1))
features.Similarity_Materie_Consumo = scaler.fit_transform(np.array(features.Similarity_Materie_Consumo).reshape(-1, 1))
features.Similarity_Merci = scaler.fit_transform(np.array(features.Similarity_Merci).reshape(-1, 1))
features.Similarity_Merci_Prodotti = scaler.fit_transform(np.array(features.Similarity_Merci_Prodotti).reshape(-1, 1))

X_train = features.loc[540:,]
X_test = features.loc[:539,]
y_train = targets.loc[540:,]
y_test = targets.loc[:539,]
"""

clf = RandomForestClassifier(n_estimators=100, max_depth=30, bootstrap = False)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_test, y_pred))
#print(clf.feature_importances_)

"""
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
#print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))
"""

#feature selection

model = SelectFromModel(clf, prefit=True)
features_new = pd.DataFrame(model.transform(features))
print(features_new.shape)

X_train_new, X_test_new, y_train, y_test = train_test_split(features_new, targets, test_size=0.3, shuffle=False)
clf2 = RandomForestClassifier(n_estimators=100, max_depth=50, bootstrap = False)
y_pred2 = clf2.fit(X_train_new, y_train).predict(X_test_new)
print(accuracy_score(y_test, y_pred2))

#le performance, in quanto ad accuracy, sono paragonabili tra il modello con con tutte le features e quello ridotto e intorno 
#a 0.94/0.95. E' stata testata anche una pipeline che prevede prima una standardizzazione dei dati ma senza riportare
#significative variazioni nella performance. Anche F-score assume più o meno gli stessi valori (nella versione weighted che tiene
#conto della dimensione delle classi). Invece la versione macro è potenzialmente anche più alta dell'accuracy perchè, come si 
#vede dalla confusion matrix, gli errori sono di solito sulle classe più grandi; a volte però si abbassa quando non viene appresa la classe
#66/25/509, che ha poche istanze, e quindi il suo F score è 0.


0.9517625231910947
(1794, 142)


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:444: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  f"X has feature names, but {self.__class__.__name__} was fitted without"


0.9443413729128015


In [56]:
#salvataggio del modello RF fittato sui dati supervised di test finora disponibili così da poterlo usare nel Notebook 2

import pickle

#clf = RandomForestClassifier(n_estimators=100, max_depth=30, bootstrap = False)
#clf.fit(X_train, y_train)

filename = '/content/finalized_model.sav'
pickle.dump(clf, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.9517625231910947


In [161]:
#feature selection basata sul random forest, basata sulle features scelte inizialmente

mask = clf.feature_importances_ > 0.01
print(features.columns[mask])

#Risultano importanti il prezzo unitario, l'IVA, il giorno (???), la modalità di pagamento, il codice tipo e il codice cedente
#Risultano poco importanti il prezzo totale, il mese, il numero DDT e alcune features estratte dalla descrizione. Vuol dire che 
#non hanno una grande influenza sulle performance ma comunque aiutano un po' il modello.
#Risultano non importanti il codice cessionario, il tipo di documento, la condizione di pagamento e l'anno. In effetti queste features assumono sempre (o quasi)
#lo stesso valore in questo dataset (eliminate)


Index(['PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA', 'Month', 'Day',
       'CodiceTipo_', 'CodiceTipo_AswArtFor', 'CodiceTipo_Codice',
       'CodiceTipo_Codice articolo fornitore', 'CodiceTipo_EAN',
       'ModalitaPagamento_', 'ModalitaPagamento_MP01',
       'ModalitaPagamento_MP02', 'ModalitaPagamento_MP05',
       'ModalitaPagamento_MP19', 'CodiceCedente_10483110010',
       'CodiceCedente_183410653', 'CodiceCedente_4352551214',
       'CodiceCedente_6909241215', 'CodiceCedente_7581111213',
       'CodiceCedente_777280157', 'CodiceCedente_9407411215', 'NumeroDDT_',
       'NumeroDDT_0848053410', 'it', 'litro', 'mat', 'spese', 'varie'],
      dtype='object')


In [177]:
#quarto classificatore provato: AdaBoost

from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=8), n_estimators=100)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_pred, y_test))

"""
from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))
"""

#accuracy intorno a 0.94/0.95, come RF, però non funziona la feature selection.
#Osservazione: l'accuracy sale fino a 0.96/0.97 se la dimensione del training viene aumentata (90% dei dati) a scapito di quella del test set, cosa che 
#non accade nel RF

0.9443413729128015


"\nfrom sklearn.metrics import f1_score\nprint(f1_score(y_test, y_pred, average=None))\nprint(f1_score(y_test, y_pred, average='micro'))\nprint(f1_score(y_test, y_pred, average='macro'))\nprint(f1_score(y_test, y_pred, average='weighted'))\n"

In [ ]:
#quinto classificatore provato: SVC

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

features = data_labeled.drop(['Conto'], axis=1)

"""
scaler = StandardScaler()
features.PrezzoUnitario = scaler.fit_transform(np.array(features.PrezzoUnitario).reshape(-1, 1))
features.PrezzoTotale = scaler.fit_transform(np.array(features.PrezzoTotale).reshape(-1, 1))
features.AliquotaIVA = scaler.fit_transform(np.array(features.AliquotaIVA).reshape(-1, 1))
features.Year = scaler.fit_transform(np.array(features.Year).reshape(-1, 1))
features.Month = scaler.fit_transform(np.array(features.Month).reshape(-1, 1))
features.Day = scaler.fit_transform(np.array(features.Day).reshape(-1, 1))
"""

print(features.columns)
targets = data_labeled.Conto
#print(features.shape)

X_train, X_test, y_train, y_test = train_test_split(features, targets, test_size=0.3, shuffle=False)
clf = make_pipeline(StandardScaler(), SVC(gamma='scale'))
y_pred = clf.fit(X_train, y_train).predict(X_test)
print(accuracy_score(y_pred, y_test))

from sklearn.metrics import f1_score
print(f1_score(y_test, y_pred, average=None))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='weighted'))

#performance intorno a 0.87, anche al variare di alcuni parametri come il coefficiente di regolarizzazione C e gamma.
#Non cambia praticamente niente se si normalizzano tutte le feature o solo quelle numeriche.
#Nei 2 modelli migliori (AdaBoost e RF) l'accuracy e F score sono allineati, nei modelli peggiori come appunto SVC, F score
#in modalità macro risente del fatto che alcune classi non vengono apprese e perciò hanno F score individuale uguale a 0.

Index(['PrezzoUnitario', 'PrezzoTotale', 'AliquotaIVA',
       'Similarity_Materie_Prime', 'Similarity_Materie_Consumo',
       'Similarity_Merci', 'Similarity_Merci_Prodotti', 'Year', 'Month', 'Day',
       ...
       'NumeroDDT_PKL.21.1321', 'NumeroDDT_PKL.21.1457',
       'NumeroDDT_PKL.21.161', 'NumeroDDT_PKL.21.195', 'NumeroDDT_PKL.21.471',
       'NumeroDDT_PKL.21.520', 'NumeroDDT_PKL.21.55', 'NumeroDDT_PKL.21.597',
       'NumeroDDT_PKL.21.914', 'NumeroDDT_PKL.21.97'],
      dtype='object', length=204)
0.8534322820037106
[0.87826087 0.88297872 0.9044586  0.81506849 0.         0.
 0.         1.         0.82758621 0.99130435 1.        ]
0.8534322820037106
0.6636052035971737
0.8332993756936149


In [162]:
#controllo di quali classi sono presenti nel train set, nel test set e quali vengono effettivamente predette

import numpy as np

print(np.unique(y_train))
print(np.unique(y_test))
print(np.unique(y_pred))

['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/507'
 '66/25/509' '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490'
 '70/05/101']
['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/509'
 '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490']
['66/05/006' '66/20/005' '66/25/005' '66/25/006' '66/25/506' '66/25/507'
 '66/25/509' '66/30/015' '66/30/017' '66/30/055' '68/05/025' '68/05/490']


In [ ]:
#controllo della numerosità delle varie classi nei set
res = {}
unique, counts = np.unique(y_train, return_counts=True)
for i,j in zip(unique, counts):
  res[i] = {'train' : j}

unique, counts = np.unique(y_test, return_counts=True)
for i,j in zip(unique, counts):
  res[i]['test'] = j

unique, counts = np.unique(y_pred, return_counts=True)
for i,j in zip(unique, counts):
  res[i]['pred'] = j

for k,v in res.items():
  print(k,v)


66/05/006 {'train': 256, 'test': 121, 'pred': 121}
66/20/005 {'train': 166, 'test': 89, 'pred': 90}
66/25/005 {'train': 200, 'test': 86, 'pred': 74}
66/25/006 {'train': 285, 'test': 129, 'pred': 142}
66/25/506 {'train': 14, 'test': 20, 'pred': 20}
66/25/507 {'train': 13}
66/25/509 {'train': 6, 'test': 2, 'pred': 2}
66/30/015 {'train': 17, 'test': 6, 'pred': 4}
66/30/017 {'train': 5, 'test': 3, 'pred': 3}
66/30/055 {'train': 39, 'test': 24, 'pred': 24}
68/05/025 {'train': 236, 'test': 57, 'pred': 57}
68/05/490 {'train': 6, 'test': 2, 'pred': 2}
70/05/101 {'train': 12}


In [163]:
#confusion matrix per analisi degli errori piu comuni
import numpy as np
from sklearn.metrics import confusion_matrix
unique, counts = np.unique(y_test, return_counts=True)
print(dict(zip(unique, counts)))

confusion_matrix(y_test, y_pred, labels=np.unique(y_test))

#con l'aggiunta delle informazioni estratte dalla descrizione le performance migliorano ma gli errori restano relativi alle stesse
#categorie di prima, in misura minore. Fortunatamente queste features aggiuntive non creano ocnfusione nelle altre classi

{'66/05/006': 121, '66/20/005': 89, '66/25/005': 86, '66/25/006': 129, '66/25/506': 20, '66/25/509': 2, '66/30/015': 6, '66/30/017': 3, '66/30/055': 24, '68/05/025': 57, '68/05/490': 2}


array([[118,   3,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  6,  82,   0,   1,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,  76,  10,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   3, 125,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,  20,   0,   0,   0,   0,   0,   0],
       [  0,   0,   0,   0,   0,   2,   0,   0,   0,   0,   0],
       [  0,   2,   0,   0,   0,   0,   4,   0,   0,   0,   0],
       [  0,   1,   0,   0,   0,   0,   0,   2,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,  24,   0,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,  57,   0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   2]])

In [ ]:
#ora creo il dataset per il semi supervised training con le 1255 unità non usate per il test e tutte quelle con 'Conto' NaN.
#Inoltre viene performato l'encoding delle labels per non avere stringhe e -1 ma solo interi

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(data_labeled['Conto'])
data_labeled['Conto'] = le.transform(data_labeled['Conto'])
data_semi = pd.concat(objs=[data_train, data_unlabeled])
data_semi.reset_index(inplace=True)
data_semi = data_semi.drop('index', axis=1)
data_semi = data_semi.fillna(-1)
print(data_semi)


      PrezzoUnitario  PrezzoTotale  ...  NumeroDDT_PKL.21.97  Conto
0             17.980        308.16  ...                    0      2
1              0.053          1.06  ...                    0      2
2             19.000        369.36  ...                    0      3
3              0.000          0.00  ...                    0      3
4              1.350         27.00  ...                    0      3
...              ...           ...  ...                  ...    ...
5311           0.000          0.00  ...                    0     -1
5312           0.000          0.00  ...                    0     -1
5313           0.000          0.00  ...                    0     -1
5314           0.000          0.00  ...                    0     -1
5315          10.000         10.00  ...                    0     -1

[5316 rows x 205 columns]


In [ ]:
#QUESTE ULTIME 4 CELLE SONO MANTENUTE SOLO IN CASO IN CUI SERVISSE RIUSARE ALCUNE PARTI DI CODICE MA IN REALTA' LO PSEUDO-LABELING
#E' IMPLEMENTATO DIVERSAMENTE NEL NOTEBOOK 2

#ora il dataset data_semi è pronto per il semi supervised learning dato che ha dei -1 al posto dei target mancanti

from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn import preprocessing

#clf = make_pipeline(StandardScaler(), SVC(gamma='scale', probability=True))
#clf = tree.DecisionTreeClassifier(max_depth=10, min_samples_leaf=5)
clf = AdaBoostClassifier(base_estimator=tree.DecisionTreeClassifier(max_depth=8), n_estimators=100)
calibrated_clf = CalibratedClassifierCV(base_estimator=clf)

X = data_semi.drop('Conto', axis=1)
y = data_semi.Conto
calibrated_clf.fit(X, y)

self_training_model = SelfTrainingClassifier(calibrated_clf, threshold=0.95, max_iter=2)
self_training_model.fit(X, y)

SelfTrainingClassifier(base_estimator=CalibratedClassifierCV(base_estimator=AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=8),
                                                                                               n_estimators=100)),
                       max_iter=2, threshold=0.95)

In [ ]:
#controllo di quali samples sono stati labeled e a che iterazione

for i in range(len(self_training_model.labeled_iter_)):
  print(self_training_model.labeled_iter_[i])
  print(self_training_model.transduction_[i])


In [ ]:
#creazione del nuovo training set con l'aggiunta dei nuovi dati pseudo supervisionati

mask = self_training_model.labeled_iter_ != -1
data_train = data_semi.loc[mask,]
data_train.Conto = self_training_model.transduction_[mask]
print(data_train)

In [ ]:
#controllo delle performance predittive grazie all'aumento dei samples supervisionati, usando lo stesso test set
#del caso senza semi supervised learning

from sklearn.metrics import accuracy_score

print(accuracy_score(self_training_model.predict(data_test.drop('Conto', axis=1)), le.transform(data_test.Conto)))

#purtroppo le performance non migliorano come sperato a prescindere da quanto venga ampliato il training set grazie alle pseudo labels

0.9461966604823747
